In [1]:
%matplotlib inline 
from IPython.core.display import display

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

from sympy import *
init_printing()
#from math import *
from interval import interval
from interval import imath as im
import sys
from pprint import pprint

<img  src="スクリーンショット 2016-12-29 16.59.29.png"/>

<img  src="スクリーンショット 2016-12-29 19.29.13.png"/>

In [2]:
class ivmat(list):
    class DimentionNotMatchError(Exception):
        pass
    
    class NotListError(Exception):
        pass
    
    class SizeNotMatchError(Exception):
        pass
 
    def _get_shape(self, X):
        if isinstance(X, list):            
            for index, x in enumerate(X):
                if index==0 and isinstance(x, list):
                    col_num = len(x)
                elif isinstance(x, list):
                    if not col_num == len(x):
                        raise DimentionNotMatchError()
            return (len(X), col_num)
        else:
            raise NotListError()
    
    def __init__(self, args):
        self.shape = self._get_shape(args)
        self.size = reduce(lambda x,y: x*y, self.shape)
        #self.midpoint = self._get_midpoint()
        super(ivmat, self).__init__(args)
    
    def __pos__(self): # +x
        return self
    
    def __neg__(self): # -x
        return ivmat(map(lambda x_row: 
                         [-x for x in x_row], 
                         self))
    
    def __add__(self, other): # x + y
        return ivmat(map(lambda (x_row,y_row): 
                         [x+y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
        
    def __radd__(self, other): # y + x
        return ivmat(map(lambda (x_row,y_row): 
                         [x+y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
    def __sub__(self, other): # x - y
        return ivmat(map(lambda (x_row,y_row): 
                         [x-y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
    def __mul__(self, other): # x * y
        return ivmat(map(lambda (x_row,y_row): 
                         [x*y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
        
    def __rmul__(self, other): # y * x
        return ivmat(map(lambda (x_row,y_row): 
                         [x*y for x,y in zip(x_row, y_row)], 
                         zip(self, other)))
    
    @classmethod
    def dot(self, x, y): # matrix operation
        if not len(x[0]) == len(y):
            raise DimentionNotMatchError('dot dimention unmatch')
        mat = ivmat([[None for col in range(len(y[0]))] for row in range(len(x))])
        for i in range(len(x)):
            for j in range(len(y[0])):
                sum = 0
                for k in range(len(x[0])):
                    sum += x[i][k] * y[k][j]
                mat[i][j] = sum
        return mat
    
    @property
    def midpoint(self):
        mat = ivmat([[None for col in range(self.shape[1])] for row in range(self.shape[0])])
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                mat[i][j] = self[i][j].midpoint
        return mat
    
    def _flatten(self): # 1-D
        return reduce(lambda x,y: x+y, self)
    
    def reshape(self, row_num, col_num):
        if not row_num * col_num == self.size:
            raise SizeNotMatchError('reshape method cannnot change elements size')    
        mat = ivmat([[None for col in range(col_num)] for row in range(row_num)])
        for i, x in enumerate(self._flatten()):
            row_index = i // col_num
            col_index = i % col_num
            mat[row_index][col_index] = x
        return mat
    
    
    
a = ivmat([[interval[-2,1]], [interval[3,4]], [interval[2,3]]])
b = ivmat([[interval[-5,-4]], [interval[-3,1]], [interval[-1,2]]])
c = ivmat([[interval[2,3], interval[5,7], interval[0,1]]])
d = ivmat([[interval[1,2], interval[-2,1], interval[4,6]]])

from itertools import chain

print b
print a+b
print a-b
print a*b
print
print a
print(a.midpoint)

[[interval([-5.0, -4.0])], [interval([-3.0, 1.0])], [interval([-1.0, 2.0])]]
[[interval([-7.0, -3.0])], [interval([-0.0, 5.0])], [interval([1.0, 5.0])]]
[[interval([2.0, 6.0])], [interval([2.0, 7.0])], [interval([-0.0, 4.0])]]
[[interval([-5.0, 10.0])], [interval([-12.0, 4.0])], [interval([-3.0, 6.0])]]

[[interval([-2.0, 1.0])], [interval([3.0, 4.0])], [interval([2.0, 3.0])]]
[[interval([-0.5])], [interval([3.5])], [interval([2.5])]]


In [3]:
tmp = ivmat.dot(a,c)

pprint(tmp)
print type(tmp)
print tmp.shape
print tmp.size

pprint(tmp.reshape(9,1))

[[interval([-6.0, 3.0]), interval([-14.0, 7.0]), interval([-2.0, 1.0])],
 [interval([6.0, 12.0]), interval([15.0, 28.0]), interval([0.0, 4.0])],
 [interval([4.0, 9.0]), interval([10.0, 21.0]), interval([0.0, 3.0])]]
<class '__main__.ivmat'>
(3, 3)
9
[[interval([-6.0, 3.0])],
 [interval([-14.0, 7.0])],
 [interval([-2.0, 1.0])],
 [interval([6.0, 12.0])],
 [interval([15.0, 28.0])],
 [interval([0.0, 4.0])],
 [interval([4.0, 9.0])],
 [interval([10.0, 21.0])],
 [interval([0.0, 3.0])]]


In [15]:
x_1, x_2, x_3 = symbols("x_1 x_2 x_3")
f_1 = symbols("f_1", cls=Function)
f_2 = symbols("f_2", cls=Function)
f_1 = x_1**2 + x_2**2 - 1
f_2 = x_1 - x_2**2
f = [[lambda x_1,x_2: str(f_1)], [lambda x_1,x_2: str(f_2)]]

display(f_1)
display(f_2)

class fmat(list):
    def apply_args(self,*args):
        ans = []
        for f_row in self:
            tmp = []
            for f in f_row:
                tmp.append(f(args[0], args[1]))
            ans.append(tmp)
        return ivmat(ans)
            
f_grad = fmat()
f_grad_symbol = []
for f in ['f_1', 'f_2']:
    tmp = []
    tmp_symbol = []
    for x in ['x_1', 'x_2']:
        grad_str = str(eval(f).diff(eval(x)))
        display(eval(grad_str))
        grad = lambda x_1, x_2: eval(grad_str)
        print grad(interval[0.5,0.8], interval[0.6,0.9])
        tmp.append(grad)
        tmp_symbol.append(eval(grad_str))
    f_grad.append(tmp)
    f_grad_symbol.append(tmp_symbol)
    
display(f_grad_symbol)
display(f_grad)

print '---なぜ同じ結果に？---'*2  
print f_grad[0][0](interval[0.5,0.8], interval[0.6,0.9])
print f_grad[0][1](interval[0.5,0.8], interval[0.6,0.9])
print f_grad[1][0](interval[0.5,0.8], interval[0.6,0.9])
print f_grad[1][1](interval[0.5,0.8], interval[0.6,0.9])
print '------'*10

"""
このようにそれぞれを作るとうまくいく
"""

print "別々に作るとうまくいく"
f1x1_str = str(f_1.diff(x_1)) 
f1x1 = lambda x_1, x_2: eval(f1x1_str)
f1x2_str = str(f_1.diff(x_2)) 
f1x2 = lambda x_1, x_2: eval(f1x2_str)
f2x1_str = str(f_2.diff(x_1)) 
f2x1 = lambda x_1, x_2: eval(f2x1_str)
f2x2_str = str(f_2.diff(x_2)) 
f2x2 = lambda x_1, x_2: eval(f2x2_str)


f_grad = fmat([[f1x1, f1x2], [f2x1, f2x2]])


print f_grad.apply_args(interval[0.5,0.8], interval[0.6,0.9])



interval([1.0, 1.6])


interval([1.2, 1.8])


1


interval([-1.8, -1.2])


[[<function __main__.<lambda>>, <function __main__.<lambda>>],
 [<function __main__.<lambda>>, <function __main__.<lambda>>]]

---なぜ同じ結果に？------なぜ同じ結果に？---
interval([-1.8, -1.2])
interval([-1.8, -1.2])
interval([-1.8, -1.2])
interval([-1.8, -1.2])
------------------------------------------------------------
別々に作るとうまくいく
[[interval([1.0, 1.6]), interval([1.2, 1.8])], [1, interval([-1.8, -1.2])]]


In [5]:
X = ivmat([[interval[0.5,0.8]], [interval[0.6,0.9]]])
y = X.midpoint
print y
print type(y)
Y = [[0.43,0.43], [0.29,-0.37]]




[[interval([0.65])], [interval([0.75])]]
<class '__main__.ivmat'>


In [6]:
print list(interval[2,3])
print list([interval[2,3]])
print [interval[2,3]]

[(2.0, 3.0)]
[interval([2.0, 3.0])]
[interval([2.0, 3.0])]
